In [8]:

import json
import numpy as np
from son_main_script import Son
from son_pymoo import HighRssiFirstSampling, ObjectiveEnum, SonProblemElementWise
from pymoo.decomposition.asf import ASF
import plotly.graph_objects as go
from pymoo.indicators.hv import Hypervolume

def asf_select(objective_space: np.ndarray,
                    weights=[]):
    
    # TODO check if picked solution is from pareto front
    approx_ideal = objective_space.min(axis=0)
    approx_nadir = objective_space.max(axis=0)

    np.seterr(divide='ignore', invalid='ignore')
    nF = (objective_space - approx_ideal) / (approx_nadir - approx_ideal)
    decomp = ASF()

    if len(weights) != objective_space.shape[1]:
        weights = [1/objective_space.shape[1] for _ in range(objective_space.shape[1])]

    weights_np= np.array(weights)
    i = decomp.do(nF, weights_np).argmin()
    return i

def normalize_objective_space_pop(objective_space_pop: np.ndarray, ideal, nadir):
    return (objective_space_pop - ideal) / (nadir - ideal)

def create_greedy_rssi_sampling_pop(optimization_objectives: list[str], son: Son):
     ###### test HighRssiFirstSampling
    son_problem_obj = SonProblemElementWise(optimization_objectives, son)
    # create pop with RssiSampling
    high_rssi_sampling_obj = HighRssiFirstSampling()
    sampling_pop = high_rssi_sampling_obj._do(son_problem_obj, 100)
    sampling_pop_objecitve_space = np.empty((100,2))
    out = {}
    for index, ind in enumerate(sampling_pop):
        son_problem_obj._evaluate(ind, out)
        sampling_pop_objecitve_space[index]=out["F"]
    return sampling_pop_objecitve_space

def get_hypervolume(objective_space_pop: np.ndarray, ref_point=None, norm_ref_point=True, zero_to_one=False, ideal=None, nadir=None):
    hv = Hypervolume(ref_point=np.array([1.1, 1.1]),
                     norm_ref_point=norm_ref_point,
                     zero_to_one=zero_to_one,
                     ideal=ideal,
                     nadir=nadir)
    
    return hv(objective_space_pop)

##################################################################

# network_name = "hetNet2"
# config_names = ["static600gen_pop200_opff40", "static600gen_pop400_off80"]

# config_names= ["efficiency_static"]

network_name = "hetNet4"
config_names = ["static_randomsampling", "static_greedysampling"]
# config_names = ["static_greedysampling2"]

# network_name = "hetNet4"
# config_names =  ["static_energyConsumption"]

##################################################################
pareto_fronts = []
hyper_volume_histories = []

for _, config_name in enumerate(config_names):
    results_directory = f"./datastore/{network_name}/{config_name}/"

    results_object = {}
    with open(results_directory + "objectives_result.json", mode="r",encoding="utf-8") as openfile:
            results_object = json.load(openfile)

    pareto_fronts.append(results_object["objectiveSpace"])
    
    hyper_volume_history = []


    combined_his = []
    for _, current_pop_objectivespace in enumerate(results_object["history"]["objective_space_opt"]):
         combined_his.append(current_pop_objectivespace[0])

    nadir = np.array(combined_his).max(axis=0)
    ideal = np.array(combined_his).min(axis=0)
         
        
    for _, current_pop_objectivespace in enumerate(results_object["history"]["objective_space_opt"]):
        normalized_current_pop = normalize_objective_space_pop(np.array(current_pop_objectivespace), ideal, nadir)
        hyper_volume_history.append(get_hypervolume(np.array(normalized_current_pop)))
        
    hyper_volume_histories.append(hyper_volume_history)

In [9]:
fig1 = go.Figure()
fig1.update_layout(
    title=f"pareto fronts",
    xaxis_title="total energy energy consumption",
    yaxis_title="avg downlink")

##### add paretofronts of all runs
for index, pareto_front in enumerate(pareto_fronts):
    fig1.add_trace(
        go.Scatter(
            x=np.array(pareto_front)[:,0],
            y=np.array(pareto_front)[:,1],
            mode='markers',
            name=config_names[index],
            )
        )

##### add rssi sampling population
son = Son(adjacencies_file_name=f"./datastore/{network_name}/{network_name}_adjacencies.json", parameter_config_file_name=f"./datastore/{network_name}/{config_names[0]}/{config_names[0]}.json")
sampling_pop_objecitve_space = create_greedy_rssi_sampling_pop(results_object["optimization_objectives"], son)
fig1.add_trace(
     go.Scatter(
        x=sampling_pop_objecitve_space[:,0],
        y=sampling_pop_objecitve_space[:,1],
        mode='markers',
        name= "greedy_sampling_pop",
        )
    )
###### highlight greedy assignment activation profile
fig1.add_trace(
    go.Scatter(
        x=np.array(sampling_pop_objecitve_space[-1][0]),
        y=np.array(sampling_pop_objecitve_space[-1][1]),
        mode='markers',
        name= "greedy_assignment_profile",
        opacity= 0.5,
        marker=dict(size=10)
        )
    )
##### highlight asf selection from rssi_sampling
i = asf_select(sampling_pop_objecitve_space)
fig1.add_trace(
    go.Scatter(
        x=np.array(sampling_pop_objecitve_space[i][0]),
        y=np.array(sampling_pop_objecitve_space[i][1]),
        mode='markers',
        name= "asf selection for greedy_sampling_pop",
        opacity= 0.5,
        marker=dict(size=10)
        )
    )

fig2 = go.Figure()
fig2.update_layout(
    title="hypervolume over n_gen",
    xaxis_title="generation",
    yaxis_title="hypervolume")

for index, hyper_volume_history in enumerate(hyper_volume_histories):
    fig2.add_trace(
        go.Scatter(
            x=list(range(0, len(hyper_volume_history))),
            y=hyper_volume_history,
            mode='lines',
            name=config_names[index],
            ))


fig1.show()
fig2.show()

[0.5 0.5]
